In [10]:
#
# classifer notebook
#
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Lambda, Compose
from LandmarkDataset import LandmarkDataset

#
# MODEL
#

class NN(torch.nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, num_classes)
        #self.fc2 = torch.nn.Linear(hidden_size, 48)
        #self.fc3 = torch.nn.Linear(48, num_classes)
        self.dropout = torch.nn.Dropout(p=0.05)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out)
        
        out = self.fc2(out)
        #out = self.relu(out)
        #out = self.dropout(out)
        
        #out = self.fc3(out)
        
        return out

    
#
# PARAMS
#

batch_size = 4

input_size = 2 * (21 * 3) + 12 #138
hidden_size = 96
num_classes = 17   # LOOKIE!

learning_rate = .01
num_epochs = 250


#
# INITIAL DATA
#

transformations = Compose([
    Lambda(lambda x: torch.tensor(x.values).float())
])

dataset = LandmarkDataset("/home/jovyan/train/data/20230119_landmarks_reset_class_merged.csv",
                              transform=transformations)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

#
# INIT MODELS, LOSS FN, GRAD
#

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NN(input_size, hidden_size, num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

#
# TRAIN
#

running_loss_epoch = 0
for epoch in range(num_epochs):
    for batch_idx, (labels, landmarks) in enumerate(dataloader):  

        # zero out accumulated gradients
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(landmarks)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        # NB: len(dataloader) is num of batches
        running_loss_epoch += loss.item()
        if ((epoch+1) % 10 == 0) and ((batch_idx+1) % len(dataloader) == 0):
            print ('Epoch [{}/{}], Loss: {:.4f}, Running Loss (per N epoch): {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item(), running_loss_epoch))
            running_loss_epoch = 0


Epoch [10/250], Loss: 1.3887, Running Loss (per N epoch): 5195.1718
Epoch [20/250], Loss: 0.6158, Running Loss (per N epoch): 3683.0187
Epoch [30/250], Loss: 0.8362, Running Loss (per N epoch): 3201.6326
Epoch [40/250], Loss: 0.3396, Running Loss (per N epoch): 2957.9915
Epoch [50/250], Loss: 0.6232, Running Loss (per N epoch): 2732.4332
Epoch [60/250], Loss: 0.0047, Running Loss (per N epoch): 2667.2595
Epoch [70/250], Loss: 0.5422, Running Loss (per N epoch): 2560.3196
Epoch [80/250], Loss: 0.5605, Running Loss (per N epoch): 2577.2741
Epoch [90/250], Loss: 0.2069, Running Loss (per N epoch): 2648.5599
Epoch [100/250], Loss: 0.0639, Running Loss (per N epoch): 2676.1245
Epoch [110/250], Loss: 0.7210, Running Loss (per N epoch): 2662.7675
Epoch [120/250], Loss: 0.3607, Running Loss (per N epoch): 2616.8140
Epoch [130/250], Loss: 0.5081, Running Loss (per N epoch): 2762.4470
Epoch [140/250], Loss: 1.3687, Running Loss (per N epoch): 2652.6873
Epoch [150/250], Loss: 0.2608, Running Loss

In [11]:
#
# EXPORT
#

dummy_input = torch.zeros(input_size)
#model.load_state_dict(torch.load('./model_overfit.pt'))
torch.onnx.export(model, dummy_input, 'onnx_model.onnx', export_params=True,
                  input_names = ['landmarks'], output_names = ['class'], verbose=True)

#import onnx
#onnx_model = onnx.load("./onnx_model.onnx")
#onnx.checker.check_model(onnx_model)


Exported graph: graph(%landmarks : Float(138, strides=[1], requires_grad=0, device=cpu),
      %fc1.bias : Float(96, strides=[1], requires_grad=1, device=cpu),
      %fc2.bias : Float(17, strides=[1], requires_grad=1, device=cpu),
      %onnx::MatMul_12 : Float(138, 96, strides=[1, 138], requires_grad=0, device=cpu),
      %onnx::MatMul_13 : Float(96, 17, strides=[1, 96], requires_grad=0, device=cpu)):
  %onnx::Add_6 : Float(96, strides=[1], device=cpu) = onnx::MatMul[onnx_name="MatMul_0"](%landmarks, %onnx::MatMul_12) # /opt/conda/lib/python3.10/site-packages/torch/nn/modules/linear.py:114:0
  %input : Float(96, strides=[1], requires_grad=1, device=cpu) = onnx::Add[onnx_name="Add_1"](%fc1.bias, %onnx::Add_6) # /opt/conda/lib/python3.10/site-packages/torch/nn/modules/linear.py:114:0
  %input.3 : Float(96, strides=[1], requires_grad=1, device=cpu) = onnx::Relu[onnx_name="Relu_2"](%input) # /opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1453:0
  %onnx::Add_10 : Float(17, 

In [12]:
#
# EVAL
#
torch.set_printoptions(precision=4, sci_mode=False)
softmax = torch.nn.Softmax(dim=1)
accuracy = 0
count = 0
with torch.no_grad():
    
    for batch_idx, (labels, landmarks) in enumerate(dataloader):  
        out = model(landmarks)
        prob = softmax(out.data)     #setup for threshold or 'garbage' class
        _, klass = torch.max(out.data, 1)
        
        print(klass, labels, klass==labels)
        print(prob)
        print("-----")
        
        accuracy += (klass == labels).sum().item()
        count += len(labels)
        

print('--------')
print("Accuracy {}/{} : {:.4f}".format(accuracy, count, accuracy/count))


tensor([11,  6, 16, 15]) tensor([11,  6, 10, 16]) tensor([ True,  True, False, False])
tensor([[    0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
             0.0000,     0.0000,     0.0000,     0.0000,     0.0005,     0.9556,
             0.0000,     0.0041,     0.0385,     0.0013,     0.0000],
        [    0.0000,     0.0000,     0.0001,     0.0001,     0.0083,     0.0000,
             0.6102,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
             0.0000,     0.0001,     0.0001,     0.3406,     0.0405],
        [    0.0000,     0.0000,     0.0236,     0.0000,     0.0000,     0.0000,
             0.0001,     0.0000,     0.0000,     0.0019,     0.2281,     0.0000,
             0.0638,     0.0156,     0.0006,     0.2317,     0.4345],
        [    0.0000,     0.0000,     0.0000,     0.0000,     0.0113,     0.0000,
             0.0000,     0.0000,     0.0000,     0.0000,     0.0201,     0.0000,
             0.0000,     0.0000,     0.0000,     0.5310

In [13]:
print("Done")

Done
